# Quora Question Pairs


In this project,i will build regression model to predict probability duplicates of question pairs .We will use the [Quora Question Pairs](https://www.kaggle.com/c/quora-question-pairs/data) dataset from Kaggle

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from gensim import models

G:\Temp\TxGameDownload\New folder\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
data=pd.read_csv('train.csv')

In [3]:
data.shape

(404290, 6)

In [4]:
data.head(30)

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0
5,5,11,12,Astrology: I am a Capricorn Sun Cap moon and c...,"I'm a triple Capricorn (Sun, Moon and ascendan...",1
6,6,13,14,Should I buy tiago?,What keeps childern active and far from phone ...,0
7,7,15,16,How can I be a good geologist?,What should I do to be a great geologist?,1
8,8,17,18,When do you use シ instead of し?,"When do you use ""&"" instead of ""and""?",0
9,9,19,20,Motorola (company): Can I hack my Charter Moto...,How do I hack Motorola DCX3400 for free internet?,0


In [5]:
data=data.drop(['id','qid1','qid2'], axis=1)

I will check how many null values in data. 

In [6]:
np.sum(pd.isnull(data),axis=0)

question1       1
question2       2
is_duplicate    0
dtype: int64

In [7]:
drop_index=[]
for i in range(data.shape[0]):
    if(any(pd.isnull(data.iloc[i]))):
        print(i)
        drop_index.append(i)

105780
201841
363362


next,i check this is balance data?

In [8]:
print('Duplicate pairs: {}%'.format(round(data['is_duplicate'].mean()*100, 2)))

Duplicate pairs: 36.92%


In [9]:
data_df=data.drop(drop_index)
np.sum(pd.isnull(data_df),axis=0)

question1       0
question2       0
is_duplicate    0
dtype: int64

In [10]:
import string

translator = str.maketrans('', '', string.punctuation)

def pre_process(text):
    return text.translate(translator).lower()
data_df["question1"] = data_df["question1"].apply(pre_process)
data_df["question2"] = data_df["question2"].apply(pre_process)


In [11]:
train_data=data_df.drop(['is_duplicate'], axis=1)

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer_1gram = TfidfVectorizer()
X1=vectorizer_1gram.fit_transform(train_data["question1"])

In [13]:
X2=vectorizer_1gram.transform(train_data["question2"])

In [14]:
labels=data_df['is_duplicate']

In [15]:
m=int(X1.shape[0]*0.8)
X1_train=X1[:m]
X2_train=X2[:m]
X1_test=X1[m:]
X2_test=X2[m:]
label_train=labels[:m]
label_test=labels[m:]


In [16]:
(label_test==1).sum()/len(label_test)

0.35972940216181454

In [17]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import accuracy_score,f1_score,classification_report

In [18]:
def getsimilarity(X1,X2):
    k=[]
    for i in range(X1.shape[0]):
        k.append(cosine_similarity(X1_train[i],X2_train[i])[0][0])
    return k

In [19]:
similarity=getsimilarity(X1_train,X2_train)

In [20]:
def choisethreshold(similarity,labels,metric,threshold):
    predict=[1 if i>threshold else 0 for i in similarity]
    score=metric(predict,labels)
    return score

In [21]:
for i in range(2,10):
    s=choisethreshold(similarity,label_train,accuracy_score,i/10)
    print(s,'----', i/10)

0.539061123152222 ---- 0.2
0.5993556545640619 ---- 0.3
0.6431086884602185 ---- 0.4
0.6622999174471058 ---- 0.5
0.6617495648194812 ---- 0.6
0.6531541698487149 ---- 0.7
0.6515123875719246 ---- 0.8
0.654004433739708 ---- 0.9


----------------------------------threshold=0.5------------------------------------

In [22]:
test_similarity=getsimilarity(X1_test,X2_test)

In [23]:
print(choisethreshold(test_similarity,label_test,accuracy_score,0.5))

0.5010388582453189


# TRY LogisticRegression

In [24]:
X=X1_train+X2_train
print(X.shape,len(label_train))

(323429, 80902) 323429


In [25]:
print("X1",X1_train[0])
print("X2",X2_train[0])
print("X",X[0])

X1   (0, 36489)	0.1802414779430659
  (0, 44607)	0.2815540551560502
  (0, 65228)	0.30863165756730837
  (0, 36247)	0.2057111674043878
  (0, 37769)	0.29657810733872475
  (0, 72692)	0.10369773192951218
  (0, 32234)	0.3653969018559771
  (0, 13706)	0.1997881013001491
  (0, 68697)	0.6764910254404864
  (0, 71880)	0.08084579917537567
  (0, 38068)	0.08835564823799753
  (0, 78253)	0.0809005143251242
X2   (0, 78253)	0.08363065878095738
  (0, 72692)	0.10719721262220676
  (0, 71880)	0.08357409716256792
  (0, 68697)	0.6993205245843958
  (0, 65228)	0.3190470302732927
  (0, 44607)	0.2910556417542818
  (0, 38068)	0.09133738061869764
  (0, 37769)	0.3065867096600682
  (0, 36247)	0.10632664447280289
  (0, 32234)	0.3777279276115317
  (0, 13706)	0.20653033751034147
X   (0, 78253)	0.16453117310608156
  (0, 38068)	0.1796930288566952
  (0, 71880)	0.16441989633794357
  (0, 68697)	1.3758115500248822
  (0, 13706)	0.40631843881049057
  (0, 32234)	0.7431248294675088
  (0, 72692)	0.21089494455171892
  (0, 37769)	0.60

In [26]:
X_test=X1_test+X2_test

In [27]:
from sklearn.linear_model import LogisticRegression
clf=LogisticRegression(max_iter=1000).fit(X,label_train)

In [28]:
clf.score(X,label_train)

0.7894963036709757

In [29]:
clf.score(X_test,label_test)

0.7637834227905711

# Try with 2-3 Ngram

In [30]:
vectorizer_2_3gram = TfidfVectorizer(ngram_range=(2,3))

In [31]:
X_1=vectorizer_2_3gram.fit_transform(train_data["question1"])
X_2=vectorizer_2_3gram.transform(train_data["question2"])

In [32]:
X_1.shape

(404287, 2269177)

In [33]:
X_1_train=X_1[:m]
X_2_train=X_2[:m]
X_1_test=X_1[m:]
X_2_test=X_2[m:]

In [34]:
X_=X_1_train+X_2_train

In [35]:
X_test_=X_1_test+X_2_test

In [36]:
import scipy.sparse as sp

In [37]:
X_and_X_=sp.hstack((X, X_), format='csr')

In [38]:
test_and_test_=sp.hstack((X_test,X_test_),format='csr')

In [39]:
clf_23=LogisticRegression(max_iter=2000,C=1e1).fit(X_and_X_,label_train)

In [40]:
clf_23.score(X_and_X_,label_train)

0.9931886132659719

In [41]:
clf_23.score(test_and_test_,label_test)

0.8163199683395582

In [42]:
print(classification_report(label_test,clf_23.predict(test_and_test_)))

              precision    recall  f1-score   support

           0       0.84      0.89      0.86     51771
           1       0.78      0.69      0.73     29087

    accuracy                           0.82     80858
   macro avg       0.81      0.79      0.80     80858
weighted avg       0.81      0.82      0.81     80858



In [43]:
from sklearn.svm import LinearSVC
#SVM=SVC(kernel="linear",max_iter=1000).fit(X_and_X_,label_train)

In [44]:
LinearSVC_=LinearSVC(max_iter=5000,C=1e-1).fit(X_and_X_,label_train)

In [45]:
LinearSVC_.score(X_and_X_,label_train)

0.920801783389863

In [46]:
LinearSVC_.score(test_and_test_,label_test)

0.814266986569047

In [47]:
from sklearn.model_selection import GridSearchCV
parameter_LR={'C':[1e-1,1,1e1,1e2]}
gird= GridSearchCV(LogisticRegression(max_iter=1000), parameter_LR)
gird.fit(X_and_X_,label_train)
gird.best_params_

G:\Temp\TxGameDownload\New folder\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
G:\Temp\TxGameDownload\New folder\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model

{'C': 10.0}

In [48]:
from sklearn.ensemble import VotingClassifier
vote=VotingClassifier( estimators=[('lr', clf_23), ('svm', LinearSVC_)],voting='hard')
vote.fit(X_and_X_,label_train)

VotingClassifier(estimators=[('lr',
                              LogisticRegression(C=10.0, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=2000,
                                                 multi_class='auto',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=None,
                                                 solver='lbfgs', tol=0.0001,
                                                 verbose=0, warm_start=False)),
                             ('svm',
                              LinearSVC(C=0.1, class_weight=None, dual=True,
                                        fit_intercept=True, intercept_scaling=1,
                                        loss='squared_hinge', max_iter=5000,
                

In [49]:
vote.score(test_and_test_,label_test)

0.8178411536273467

# Errors analysis

In [50]:
i=0
j=0
for i in range(len(label_test)):
    if label_test.iloc[i]!=vote.predict(test_and_test_[i]):
        if label_test.iloc[i]==1:
            i=i+1
        else:
            j=j+1
        

In [51]:
print(i,j)

80857 4139


In [52]:
index_error=[]
for i in range(len(label_test)):
    if label_test.iloc[i]!=vote.predict(test_and_test_[i]):
        index_error.append(i)

In [53]:
for i in index_error[500:600]:
    print(data_df['question1'].iloc[i+m],'-------',data_df['question2'].iloc[i+m],label_test.iloc[i])

what are the best memory boosting techniques ------- what are the best memory techniques 1
is square root of negative numbers imaginary ------- are square roots of negative numbers always imaginary 1
what is a good facebook page name for entertainment and fun ------- suggest me a funny facebook page name 1
what are the consumptive needs and activities impacts on the environment ------- what are consumptive needs and activities impacts on the environment 1
what is a good youtube channel for learning about science ------- what are the best sitesyoutube channels about science 1
when did usa and japan become allies given that the japanese bombed pearl harbour and the usa nuked japan ------- why do japan and the usa have good relations if the japanese killed thousands of innocent people at pearl harbor and the usa bombed japan with nukes 1
what is arvind kejriwals view about the ban of 500 and 1000 rupee notes ------- why is arvind kejriwal opposing scrapping of rs500 and rs1000 notes schem